![image info](./pic/41.GIF)
![image info](./pic/42.GIF)![image info](./pic/43.GIF)![image info](./pic/44.GIF)![image info](./pic/45.GIF)![image info](./pic/46.GIF)
![image info](./pic/47.GIF)

In [15]:
#p5
import numpy as np

def obj(x1, x2, x3):
    return x1 ** 2 + x2 ** 2 + x3 ** 2

def dfdx(x1, x2, x3):
    dfds = np.vstack((2 * x2, 2 * x3))
    dfdd = np.array([2 * x1], dtype=float)
    return dfds, dfdd

def st(x1, x2, x3):
    h1 = x1**2/4 +x2**2/5+x3**2/25- 1
    h2 = x1 + x2 - x3
    return h1, h2

def dhdx(x1, x2, x3):
    dhds = np.vstack((np.hstack(((2/5) * x2, (2/25) * x3)), np.array([[1, -1]], dtype=float)))
    dhdsinv = np.linalg.inv(dhds)
    dhdd = np.vstack(((1 / 2) * x1, 1.))
    return dhdsinv, dhdd, dhds

def dfdd(x1, x2, x3):
    dfds, dfdd = dfdx(x1, x2, x3)
    dhdsinv, dhdd, _ = dhdx(x1, x2, x3)
    return dfdd - np.matmul(dfds.T, np.matmul(dhdsinv, dhdd))

def hequa(x1, x2, x3):
    error = 1e-3
    h1, h2 = st(x1, x2, x3)
    h = np.vstack((h1, h2))
    hnorm = np.linalg.norm(h)
    while hnorm >= error:
        dhinv, _, dh = dhdx(x1, x2, x3)
        Lambda = 1
        ds = np.matmul(dhinv, h)
        x2 = x2 - ds[0]
        x3 = x3 - ds[1]
        h1, h2 = st(x1, x2, x3)
        h = np.vstack((h1, h2))
        hnorm = np.linalg.norm(h)
    return x1, x2, x3, hnorm

def line_search(x1, x2, x3):
    a = 1. 
    df = dfdd(x1, x2, x3)
    phi = lambda a, x1, x2, x3, df: obj(x1, x2, x3) - a * 0.3 * np.matmul(df, df.T) 
    def f_a(x1, x2, x3, a):
        df = dfdd(x1, x2, x3)
        dhdsinv, dhdd, _ = dhdx(x1, x2, x3)

        x1 = x1 - a * df.flatten()
        ds = np.matmul(np.matmul(dhdsinv, dhdd), df.T).flatten()
        x2 = x2 + a * ds[0]
        x3 = x3 + a * ds[1]
        return obj(x1, x2, x3)

    while phi(a, x1, x2, x3, df) < f_a(x1, x2, x3, a):
        a = 0.5 * a
        df = dfdd(x1, x2, x3)
    return a

eps = 1e-3  
x1 = np.array([1.0], dtype=float)
x2 = np.array([2.0], dtype=float)
x3 = np.array([3.0], dtype=float)
iter = 0 

x1, x2, x3, _ =hequa(x1, x2, x3)

dfddnorm = np.linalg.norm(dfdd(x1, x2, x3))

while dfddnorm >= eps:  
    a = line_search(x1, x2, x3)
    x1 = x1 - a * dfdd(x1, x2, x3).flatten()
    dhdsinv, dhdd, dh = dhdx(x1, x2, x3)
    ds = a * np.matmul(np.matmul(dhdsinv, dhdd), dfdd(x1, x2, x3).T).flatten()
    x2 = x2 + ds[0]
    x3 = x3 + ds[1]
    x1, x2, x3, _ = hequa(x1, x2, x3)
    dfddnorm = np.linalg.norm(dfdd(x1, x2, x3))

print(x1, x2, x3)

[-1.57329303] [1.37727404] [-0.19659334]


The answer is [-1.57329303] [1.37727404] [-0.19659334] for x